## Case study  - Uber Data Analysis

You have the uber drive data for a driver which captures the differnet aspects of driving behavior. You are a part of the project where you need to grill this data and report the important findings from the grilling and drilling exercise. You also need to provide some useful insights about the trip behaviour of a Uber driver.
Let us try to dive into the data with pandas and numpy by our side!

#### Dataset - 
There are two datasets to be used here. The dataset contains-

**Uberdrive.csv**
- Trip_Id - Id for the trip
- Start Date - the date and time of the start of the trip
- End Date - the date and time of the end of the trip
- Start Location - staring location of the trip 
- End Location  - location where the trip ended
- Purpose of drive - Purpose of the trip (Business, Personal, Meals, Errands, Meetings, Customer Support etc.)


**Uberdrive_Miles.csv**
- Trip_Id - Id for the trip
- Miles Driven  - Total miles driven between the start and the end of the trip

#### Objective-

The aim is to create a driver profile based on the below aspects on driving behavior.
- Name and number of all the unique start and stop points
- Popular start and stop points
- Rides with same start and stop points
- Starting point from which most miles have been driven
- Start- stop pairs that are most travelled in terms of distance
- busiest month in terms of number of drives and miles driven
- busiest day of the week
- peak hours
- most frequent trip category
- most frequent trip purpose
- miles driven per category and purpose
- percent composition of business miles vs personal miles
- time spent per category and purpose

----------------------
 #### Concepts To cover 
----------------------
- 1. <a href = #link1>Overview of the data at hand</a>
- 2. <a href = #link3>Filtering Data</a> 
- 3. <a href = #link2>Data profiling and the functions offered by pandas for understanding the data</a>
- 4. <a href = #link4>DateTime operations</a> 




In [1]:
# Import the libraries 
import numpy as np
import pandas as pd
from datetime import datetime

### <a id = "link1"></a>Overview of the data

In [3]:
from google.colab import files
uploaded = files.upload()

Saving uberdrive.csv to uberdrive.csv


In [7]:
from google.colab import files
uploaded = files.upload()

Saving UberDrive_Miles.csv to UberDrive_Miles.csv


In [5]:
# Read the Data 
df = pd.read_csv('uberdrive.csv')

In [ ]:
# View first 3 rows of data 
df.head(3) 

,Trip_Id,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,PURPOSE*
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies


In [8]:
df_miles = pd.read_csv('UberDrive_Miles.csv')

In [9]:
df_miles.tail(3)

,Trip_Id,MILES*
1152,1153,16.2
1153,1154,6.4
1154,1155,48.2


In [ ]:
# understand shape and size of data from Uberdrive
print(df.shape)
print(df.size)

(1155, 7)
8085


In [13]:
x = df.shape[0]
y = df.shape[1]
print(x,y)

1155 7


In [14]:
x*y

8085

In [10]:
df.size

8085

In [ ]:
# check info about data (includes column names, the number of non-null values in it, and data-type for each column.)
df.info()
print()
df_miles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 7 columns):
Trip_Id        1155 non-null int64
START_DATE*    1155 non-null object
END_DATE*      1155 non-null object
CATEGORY*      1155 non-null object
START*         1155 non-null object
STOP*          1155 non-null object
PURPOSE*       653 non-null object
dtypes: int64(1), object(6)
memory usage: 63.2+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 2 columns):
Trip_Id    1155 non-null int64
MILES*     1155 non-null float64
dtypes: float64(1), int64(1)
memory usage: 18.1 KB


In [20]:
set(df['PURPOSE*'])

{'Airport/Travel',
 'Between Offices',
 'Charity ($)',
 'Commute',
 'Customer Visit',
 'Errand/Supplies',
 'Meal/Entertain',
 'Meeting',
 'Moving',
 'Temporary Site',
 nan}

1. Some of the columns have a 1155 records while there are others with 653, why is that? Lets explore

2. We see some updates can be made in the column names, lets rename the columns


### Renaming columns

In [21]:
df.columns

Index(['Trip_Id', 'START_DATE*', 'END_DATE*', 'CATEGORY*', 'START*', 'STOP*',
       'PURPOSE*'],
      dtype='object')

In [22]:
df.columns.str.replace('','')

Index(['Trip_Id', 'START_DATE*', 'END_DATE*', 'CATEGORY*', 'START*', 'STOP*',
       'PURPOSE*'],
      dtype='object')

In [32]:
# Approach 1
# Replace the * character from all the  columns
df.columns = df.columns.str.replace("*", "")

In [33]:
# Approach 2
#
# You can also rename the specific column names 
df_miles.rename(columns = {'MILES*':'MILES'}, inplace=True)
print(df.columns,"\n", df_miles.columns)

Index(['Trip_Id', 'START_DATE', 'END_DATE', 'CATEGORY', 'START', 'STOP',
       'PURPOSE'],
      dtype='object') 
 Index(['Trip_Id', 'MILES'], dtype='object')


**The column names were updated.**

### <a id = "link3"></a>Filtering dataframes
#### Using null values

In [34]:
# shows the top 5 entries where PURPOSE is null

df[df['PURPOSE'].isnull()].head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
32,33,1/19/2016 9:09,1/19/2016 9:23,Business,Whitebridge,Lake Wellingborough,NaN
85,86,02-09-2016 10:54,02-09-2016 11:07,Personal,Whitebridge,Northwoods,NaN
86,87,02-09-2016 11:43,02-09-2016 11:50,Personal,Northwoods,Tanglewood,NaN
87,88,02-09-2016 13:36,02-09-2016 13:52,Personal,Tanglewood,Preston,NaN



#### Filtering out records based on conditions

In [36]:
# Conditions within dataframe 
df_miles[df_miles['MILES'] > 30].head()

,Trip_Id,MILES
4,5,63.7
36,37,40.2
108,109,43.7
232,233,136.0
233,234,30.2


## <a id = "link2"></a>In a bid to create the driver profile, lets explore the data parameter wise - 

- 1.Destination - (starting and stopping)

- 2.Time - (hour of the day, day of week, month of year)

- 3.Grouping two parameters to get more insights

- 4.Category and Purpose

## 1. Destination
### Understanding  the start and stop points 
###### Through the feature, we will try to understand the below points of the driver profile. 
- Name and number of all the unique start and stop points
- Popular start and stop points
- Rides with same start and stop points
- Starting point from which most miles have been driven
- Start- stop pairs that are most travelled in terms of distance

**Let us handle these one by one**

**1. Name and Number of all unique start and stop points**

In [37]:
# Get the unique starting point, unique destination
# names of unique start points
print(df['START'].unique())

['Fort Pierce' 'West Palm Beach' 'Cary' 'Jamaica' 'New York' 'Elmhurst'
 'Midtown' 'East Harlem' 'Flatiron District' 'Midtown East'
 'Hudson Square' 'Lower Manhattan' "Hell's Kitchen" 'Downtown' 'Gulfton'
 'Houston' 'Eagan Park' 'Morrisville' 'Durham' 'Farmington Woods'
 'Whitebridge' 'Lake Wellingborough' 'Fayetteville Street' 'Raleigh'
 'Hazelwood' 'Fairmont' 'Meredith Townes' 'Apex' 'Chapel Hill'
 'Northwoods' 'Edgehill Farms' 'Tanglewood' 'Preston' 'Eastgate'
 'East Elmhurst' 'Jackson Heights' 'Long Island City' 'Katunayaka'
 'Unknown Location' 'Colombo' 'Nugegoda' 'Islamabad' 'R?walpindi'
 'Noorpur Shahan' 'Heritage Pines' 'Westpark Place' 'Waverly Place'
 'Wayne Ridge' 'Weston' 'East Austin' 'West University' 'South Congress'
 'The Drag' 'Congress Ave District' 'Red River District' 'Georgian Acres'
 'North Austin' 'Coxville' 'Convention Center District' 'Austin' 'Katy'
 'Sharpstown' 'Sugar Land' 'Galveston' 'Port Bolivar' 'Washington Avenue'
 'Briar Meadow' 'Latta' 'Jacksonville'

In [38]:
print(df['START'].nunique())                           # or use can use the nunique function

177


In [39]:
# Get the names of stopping destinations, unique destinations
# Names of unique stopping points
print(df['STOP'].unique())

['Fort Pierce' 'West Palm Beach' 'Palm Beach' 'Cary' 'Morrisville'
 'New York' 'Queens' 'East Harlem' 'NoMad' 'Midtown' 'Midtown East'
 'Hudson Square' 'Lower Manhattan' "Hell's Kitchen" 'Queens County'
 'Gulfton' 'Downtown' 'Houston' 'Jamestown Court' 'Durham' 'Whitebridge'
 'Lake Wellingborough' 'Raleigh' 'Umstead' 'Hazelwood' 'Westpark Place'
 'Meredith Townes' 'Leesville Hollow' 'Apex' 'Chapel Hill'
 'Williamsburg Manor' 'Macgregor Downs' 'Edgehill Farms' 'Northwoods'
 'Tanglewood' 'Preston' 'Walnut Terrace' 'Jackson Heights' 'East Elmhurst'
 'Midtown West' 'Long Island City' 'Jamaica' 'Unknown Location' 'Colombo'
 'Nugegoda' 'Katunayaka' 'Islamabad' 'R?walpindi' 'Noorpur Shahan'
 'Heritage Pines' 'Waverly Place' 'Wayne Ridge' 'Depot Historic District'
 'Weston' 'West University' 'South Congress' 'Arts District'
 'Congress Ave District' 'Red River District' 'The Drag'
 'Convention Center District' 'North Austin' 'Coxville' 'Katy' 'Alief'
 'Sharpstown' 'Sugar Land' 'Galveston' 'Port

In [ ]:
print(len(df['STOP'].unique()))            # count of unique stopping points

188


**2a. Identify popular start points - top 10**

In [40]:
df['START'].value_counts().head(10)

Cary                201
Unknown Location    148
Morrisville          85
Whitebridge          68
Islamabad            57
Durham               37
Lahore               36
Raleigh              28
Kar?chi              27
Westpark Place       17
Name: START, dtype: int64

**2b. Identify popular stop destinations - top 10**

In [ ]:

df['STOP'].value_counts().head(10)

Cary                203
Unknown Location    149
Morrisville          84
Whitebridge          65
Islamabad            58
Durham               36
Lahore               36
Raleigh              29
Kar?chi              26
Apex                 17
Name: STOP, dtype: int64

**3.  Are there cases where the start and the stop location are the same  ?**

In [42]:
df[df['START'] == df['STOP']].head(3)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies


In [ ]:
df[df['START'] == df['STOP']].head(5)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting
5,6,01-06-2016 17:15,01-06-2016 17:19,Business,West Palm Beach,West Palm Beach,Meal/Entertain


In [ ]:
df[df['START'] == df['STOP']].shape

(288, 7)

**288 trips have the same start and stop points**

**4.Starting point from which the most miles have been driven**

**In order to use the miles feature, let us now merge the two dataframes so that the all the information is in one dataframe.**
- using merge 


In [43]:
df_join = pd.merge(df, 
                   df_miles,
                   on = 'Trip_Id',
                   how = 'left')
df_join.head(5)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7


**Let's now use groupby function to find the starting point from which the most miles have been driven**

In [52]:
df_join.groupby('START')['MILES'].sum().sort_values(ascending = False ).head(10)#.reset_index()

START
Unknown Location    1976.5
Cary                1791.3
Morrisville          671.7
Raleigh              433.0
Islamabad            401.2
Durham               384.4
Jacksonville         375.2
Latta                310.3
Asheville            287.7
Whitebridge          273.4
Name: MILES, dtype: float64

In [53]:
pd.DataFrame(df_join.groupby('START')['MILES'].sum().sort_values(ascending = False )).head(10)#.reset_index()

,MILES
START,
Unknown Location,1976.5
Cary,1791.3
Morrisville,671.7
Raleigh,433.0
Islamabad,401.2
Durham,384.4
Jacksonville,375.2
Latta,310.3
Asheville,287.7


In [57]:
pd.DataFrame(df_join.groupby('START')['MILES'].sum().sort_values(ascending = False )).head(10).reset_index()

,START,MILES
0,Unknown Location,1976.5
1,Cary,1791.3
2,Morrisville,671.7
3,Raleigh,433.0
4,Islamabad,401.2
5,Durham,384.4
6,Jacksonville,375.2
7,Latta,310.3
8,Asheville,287.7
9,Whitebridge,273.4


**5. Find the top10 start stop pair that have the most miles covered between them ever.**

In [59]:
df_join[df_join['START'] == df['STOP']].sort_values('MILES', ascending = False).iloc[0:10,:]

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
776,777,9/27/2016 21:01,9/28/2016 2:37,Business,Unknown Location,Unknown Location,NaN,195.6
727,728,8/27/2016 16:15,8/27/2016 19:13,Business,Unknown Location,Unknown Location,NaN,156.9
707,708,8/24/2016 13:01,8/24/2016 15:25,Business,Unknown Location,Unknown Location,NaN,96.2
751,752,09-06-2016 17:49,09-06-2016 17:49,Business,Unknown Location,Unknown Location,NaN,69.1
710,711,8/25/2016 17:19,8/25/2016 19:20,Business,Unknown Location,Unknown Location,NaN,50.4
979,980,11/20/2016 10:27,11/20/2016 11:32,Business,Cary,Cary,Between Offices,39.2
779,780,9/30/2016 17:39,9/30/2016 20:20,Business,Islamabad,Islamabad,NaN,37.7
708,709,8/25/2016 15:17,8/25/2016 16:22,Business,Unknown Location,Unknown Location,NaN,35.0
1089,1090,12/22/2016 15:40,12/22/2016 16:38,Business,Unknown Location,Unknown Location,Meeting,32.3
785,786,10-04-2016 09:50,10-04-2016 10:52,Business,Unknown Location,Unknown Location,NaN,28.6


# Let us drop the unknown locations
df2 = df[df['START'] != 'Unknown Location']             # Makes a new dataframe, which don't have "Unknown Location" as starting point
df2 = df2[df2['STOP'] != 'Unknown Location']            # Further updates the df2 dataframe, by removing "Unknown Location" as stopping point

In [60]:
# Let us drop the unknown locations

df2 = df_join[df_join['START'] != 'Unknown Location']             # Makes a new dataframe, which don't have "Unknown Location" as starting point

df2 = df2[df2['STOP'] != 'Unknown Location']            # Further updates the df2 dataframe, by removing "Unknown Location" as stopping point

In [64]:
# Creating a dataframe with the top 10 most miles covered between a start stop pair

k3 = df2.groupby(['START','STOP'])['MILES'].sum().sort_values(ascending=False).reset_index() # flatten the dataframe

k3 

,START,STOP,MILES
0,Morrisville,Cary,395.7
1,Cary,Durham,390.0
2,Cary,Morrisville,380.0
3,Raleigh,Cary,365.7
4,Cary,Raleigh,336.5
...,...,...,...
338,Sunnyside,Ingleside,0.7
339,Medical Centre,Tudor City,0.7
340,Central,West Berkeley,0.6
341,Soho,Tribeca,0.5


In [65]:
k3['Start-Stop'] = k3['START'] + ' - ' + k3['STOP']

In [67]:
k3 = df2.groupby(['START','STOP'])['MILES'].sum().sort_values(ascending=False).head(10) 
k3

START         STOP        
Morrisville   Cary            395.7
Cary          Durham          390.0
              Morrisville     380.0
Raleigh       Cary            365.7
Cary          Raleigh         336.5
Durham        Cary            334.4
Latta         Jacksonville    310.3
Cary          Cary            255.9
Jacksonville  Kissimmee       201.0
Asheville     Mebane          195.9
Name: MILES, dtype: float64

**The most popular start to destination pair is Morrisville-Cary**

<a id = "link4"></a>
## 2. Start Date - End Date 
### Manipulating date & time objects
#### Lets explore the variables using the below points-
- busiest month in terms of number of drives and miles driven
- busiest day of the week and preferred start hour 
- peak hours

We will create more features for the trip data to be able to cater to above profile mappings

In [68]:
df_join.head(3)

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8


In [69]:
df_join.dtypes

Trip_Id         int64
START_DATE     object
END_DATE       object
CATEGORY       object
START          object
STOP           object
PURPOSE        object
MILES         float64
dtype: object

In [70]:
# Create columns by converting the start and end date into a datatime format
# You can also over write the same column - but for the sake of understanding the difference in formats, we create new columns

df_join['start_dt'] = pd.to_datetime(df['START_DATE'])
df_join['end_dt'] = pd.to_datetime(df['END_DATE'])

In [71]:
df_join.head()             # Print first 5 rows of data.

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00


In [72]:
df_join.dtypes             # See how the dtype is different

Trip_Id                int64
START_DATE            object
END_DATE              object
CATEGORY              object
START                 object
STOP                  object
PURPOSE               object
MILES                float64
start_dt      datetime64[ns]
end_dt        datetime64[ns]
dtype: object

In [73]:
# Create more columns by using the inbuilt functionalities of datatime module

df_join['start_day'] = df_join['start_dt'].dt.day
df_join['start_hour'] = df_join['start_dt'].dt.hour
df_join['start_month'] = df_join['start_dt'].dt.month
df_join['d_of_wk'] = df_join['start_dt'].dt.dayofweek   # Days encoded as 0-6  ( monday =0, Tue =1 .... )

In [76]:
df_join.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_wk
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2


In [77]:
df_join['weekday'] = df_join['start_dt'].apply(lambda x : datetime.strftime(x,'%a'))  # ( or directly convert into the short form)

In [79]:
df_join.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_wk,weekday
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Fri
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Sat
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Sat
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tue
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wed


In [80]:
df_join['cal_month'] =  df_join['start_dt'].apply(lambda x : datetime.strftime(x,'%b'))

In [82]:
df_join.head()

,Trip_Id,START_DATE,END_DATE,CATEGORY,START,STOP,PURPOSE,MILES,start_dt,end_dt,start_day,start_hour,start_month,d_of_wk,weekday,cal_month
0,1,01-01-2016 21:11,01-01-2016 21:17,Business,Fort Pierce,Fort Pierce,Meal/Entertain,5.1,2016-01-01 21:11:00,2016-01-01 21:17:00,1,21,1,4,Fri,Jan
1,2,01-02-2016 01:25,01-02-2016 01:37,Business,Fort Pierce,Fort Pierce,NaN,5.0,2016-01-02 01:25:00,2016-01-02 01:37:00,2,1,1,5,Sat,Jan
2,3,01-02-2016 20:25,01-02-2016 20:38,Business,Fort Pierce,Fort Pierce,Errand/Supplies,4.8,2016-01-02 20:25:00,2016-01-02 20:38:00,2,20,1,5,Sat,Jan
3,4,01-05-2016 17:31,01-05-2016 17:45,Business,Fort Pierce,Fort Pierce,Meeting,4.7,2016-01-05 17:31:00,2016-01-05 17:45:00,5,17,1,1,Tue,Jan
4,5,01-06-2016 14:42,01-06-2016 15:49,Business,Fort Pierce,West Palm Beach,Customer Visit,63.7,2016-01-06 14:42:00,2016-01-06 15:49:00,6,14,1,2,Wed,Jan


**Now let us answer the questions above.**
<br>
<br>
**1. Busiest month in terms of number of drives and miles driven**

In [83]:
#groupby calender months and count the number of drives
df_join.groupby('cal_month').count()['Trip_Id'].sort_values(ascending = False)           

cal_month
Dec    146
Aug    133
Nov    122
Feb    115
Mar    113
Jul    112
Jun    108
Oct    106
Jan     61
Apr     54
May     49
Sep     36
Name: Trip_Id, dtype: int64

**December appears to be the busiest month in terms of number of drives**

In [84]:
#groupby calender months and count the number of drives
df_join.groupby('cal_month').sum()['MILES'].sort_values(ascending = False)           

cal_month
Oct    1810.0
Mar    1693.9
Aug    1335.5
Jul    1224.6
Apr    1113.0
Dec     981.3
Feb     908.2
Jun     842.8
Nov     816.9
Sep     601.8
Jan     512.9
May     363.8
Name: MILES, dtype: float64

**October appears to be the busiest month in terms of miles driven**

**2. Busiest day in terms of number of rides**

In [85]:
# Which day did the driver get most drives? 

df_join.groupby(['weekday']).size()  
#note that .count() could also have been used. However, .size() makes it look more clean.

weekday
Fri    206
Mon    174
Sat    150
Sun    148
Thu    154
Tue    176
Wed    147
dtype: int64

**3. Peak hours ?**

In [86]:
df_join.groupby('start_hour').size()             # The number of trips started for each hour.

start_hour
0     19
1      5
2      2
3      3
5      4
6      4
7     13
8     35
9     51
10    65
11    72
12    77
13    94
14    89
15    98
16    88
17    95
18    94
19    68
20    71
21    51
22    31
23    26
dtype: int64

**Looks like the peak hours seem to be between 13PM - 6PM**

#### For practice - figure out the trips that are starting and ending at the same time (0 minutes elapsed).<br>
The first step is in the cell below. Try to figure out the rest of the steps after this session. 
<br>

In [87]:
df_join['diff'] = (df_join['end_dt'] - df_join['start_dt'])

This creates a timedelta datatype

Find the date time units in  https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html

search for 'Datetime Units'

#### For practice- Exploring existing features to create new ones - Speed 
- Open for all of you to explore and figure out what all can be understood and derived from this feature

## 4. Category & Purpose
#### Explore the category and the purpose of the trips through
- Most frequent trip category
- Most frequent trip purpose
- Miles driven per category and purpose
- Percent composition of business miles vs personal miles
- time spent per category and purpose

**1. Most frequent trip category**

In [88]:
df_join['CATEGORY'].value_counts()

Business    1078
Personal      77
Name: CATEGORY, dtype: int64

**Most trips are in the business category**

**2. Most frequent Purpose**

In [89]:
#Purpose
df_join['PURPOSE'].value_counts()

Meeting            187
Meal/Entertain     160
Errand/Supplies    128
Customer Visit     101
Temporary Site      50
Between Offices     18
Moving               4
Airport/Travel       3
Commute              1
Charity ($)          1
Name: PURPOSE, dtype: int64

**Most trips are for meetings**

In [90]:
#Average distance traveled for each activity
df_join.groupby('PURPOSE').mean()['MILES'].sort_values(ascending = False)

PURPOSE
Commute            180.200000
Customer Visit      20.688119
Meeting             15.247594
Charity ($)         15.100000
Between Offices     10.944444
Temporary Site      10.474000
Meal/Entertain       5.698125
Airport/Travel       5.500000
Moving               4.550000
Errand/Supplies      3.968750
Name: MILES, dtype: float64

**Now lets try to answer some questions from this data.**

**Question3**: How many miles were driven per category and purpose ?

**Question4**: What is percentage of business miles vs personal?

**Question5**: How much time was spend for drives per category and purpose?  - <i>for practice: you will need to create a time difference variable - answers will be shared through a notebook</i>


In [ ]:
#Question3: How many miles were driven per category and purpose ?
df_join.groupby('PURPOSE').sum()['MILES'].sort_values(ascending = False)

PURPOSE
Meeting            2851.3
Customer Visit     2089.5
Meal/Entertain      911.7
Temporary Site      523.7
Errand/Supplies     508.0
Between Offices     197.0
Commute             180.2
Moving               18.2
Airport/Travel       16.5
Charity ($)          15.1
Name: MILES, dtype: float64

In [ ]:
#Question3: How many miles were driven per category and purpose ?
df_join.groupby('CATEGORY').sum()['MILES'].sort_values(ascending = False)

CATEGORY
Business    11487.0
Personal      717.7
Name: MILES, dtype: float64

In [ ]:
#Question4: What is percentage of business miles vs personal?
df1 = df_join.groupby('CATEGORY').agg({'MILES':'sum'})
df1
df1.apply(lambda x: x/x.sum()*100).rename(columns = {'MILES':'% of Miles'})

,% of Miles
CATEGORY,
Business,94.119479
Personal,5.880521


## Profile Report - 
Through the exercise, we discussed the various aspects the driver profile for the uberdriver data given. The insights received were-

**Name and number of all the unique start and stop points**<br>
We found the unique start and stop points for the driver. We figured out the localities the driver is active in.

**Popular start and stop point**<br>
Cary has been the most popular start and stop point

**Rides with same start and stop points**<br>
288 such rides were found

**Starting point from which most miles have been driven**<br>
Unknown location followed by Cary

**Start- stop pairs that are most travelled in terms of distance**<br>
Morissville - Cary

**busiest month in terms of number of drives and miles driven**<br>
In terms of no of drives - December
In terms of miles driven - October

**busiest day of the week**<br>
Friday

**peak hours**<br>
1PM - 6PM

**most frequent trip category**<br>
Business

**most frequent trip purpose**<br>
Meeting

**miles driven per category and purpose**<br>
We figured these numbers out. Category wise Business and purpose wise meetings were leading in terms of miles driven

**percent composition of business miles vs personal miles**<br>
Business - 94%
Personal - 6%


## Summary  -
Through this exercise, we tried to check out the data analysis toolkit offered by pandas. We went to explore variables at hand, use groupby, implement datatime manipulation, explored possibility to create new features and various other operations on pandas dataframe.
We also had a sneak peek into the upcoming week's topic of visualization.

Learners are recommended to explore further on this building on the points discussed in the notebook.
Happy Learning!